In [3]:
from boto3.session import Session
# import main as rst

region = 'us-east-1'
profile_name = 'cristal-management'
# aws sso login --profile cristal-management
session = Session(profile_name=profile_name, region_name=region)

In [4]:
import os

def get_aws_account_id():
    # Assuming AWS credentials are configured in the environment or via config files
    sts_client = session.client('sts')
    account_id = sts_client.get_caller_identity()['Account']
    return account_id

def set_tf_environment_id(aws_default_region):
    aws_account_id = get_aws_account_id()
    return f"{aws_account_id}-{aws_default_region}"

# def hash_environment_id(tf_environment_id, hash_command='sha1sum'):
#     # Use the provided hash command or default to sha1sum
#     result = subprocess.run([hash_command], input=tf_environment_id.encode(), capture_output=True, text=True)
#     hashed_environment_id = result.stdout.split()[0]
#     return hashed_environment_id

import hashlib

def hash_environment_id(tf_environment_id):
    # Create a SHA-1 hash object
    hash_object = hashlib.sha1()
    # Update the hash object with the bytes of the string, encoding needed to convert str to bytes
    hash_object.update(tf_environment_id.encode('utf-8'))
    # Get the hexadecimal representation of the digest
    hashed_environment_id = hash_object.hexdigest()
    return hashed_environment_id


def write_backend_config(aws_default_region, hashed_environment_id):
    backend_config = f"""terraform {{
  backend "s3" {{
    bucket         = "terraform-state-{hashed_environment_id}"
    key            = "terraform/main/main.tfstate"
    region         = "{aws_default_region}"
    encrypt        = true
    dynamodb_table = "terraform-state-lock-{hashed_environment_id}"
  }}
}}
"""
    with open("backend.tf", "w") as f:
        f.write(backend_config)

def write_tf_backend():
    aws_account_id = get_aws_account_id()
    aws_default_region = os.getenv("AWS_DEFAULT_REGION", region)
    env_id = hash_environment_id(f"{aws_account_id}-{aws_default_region}")


def main():
    # Check for AWS_DEFAULT_REGION in environment variables
    aws_default_region = os.getenv("AWS_DEFAULT_REGION", region)
    if not aws_default_region:
        print("Define env variable AWS_DEFAULT_REGION (should be your region name, ex: us-east-1) and try again.")
        exit(1)

    tf_environment_id = set_tf_environment_id(aws_default_region)
    hashed_environment_id = hash_environment_id(tf_environment_id)
    write_backend_config(tf_environment_id, aws_default_region, hashed_environment_id)

if __name__ == "__main__":
    main()
